In [128]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from random import randrange

In [129]:
#imgHotelRight/Left G-Grey
#imgShanghaiRight/Left G-Grey
#if the name contains Hl its for the hotel
#if the name contains Sh its for Shanghai
imgHlR = cv2.imread("rightHl.png")
imgHlRG = cv2.cvtColor(imgHlR,cv2.COLOR_BGR2GRAY)
imgHlL = cv2.imread("leftHl.png")
imgHlLG = cv2.cvtColor(imgHlL,cv2.COLOR_BGR2GRAY)
imgShR = cv2.imread("rightSh.png")
imgShRG = cv2.cvtColor(imgShR,cv2.COLOR_BGR2GRAY)
imgShL = cv2.imread("leftSh.png")
imgShLG = cv2.cvtColor(imgShL,cv2.COLOR_BGR2GRAY)

In [130]:
#SIFT keypoints and descriptors
sift = cv2.xfeatures2d.SIFT_create()
kpHl1, desHl1 = sift.detectAndCompute(imgHlRG,None)
kpHl2, desHl2 = sift.detectAndCompute(imgHlLG,None)
kpSh1, desSh1 = sift.detectAndCompute(imgShRG,None)
kpSh2, desSh2 = sift.detectAndCompute(imgShLG,None)

In [131]:
#k nearest neighbours
bf = cv2.BFMatcher()
matchesHl = bf.knnMatch(desHl1,desHl2, k=2)
matchesSh = bf.knnMatch(desSh1,desSh2, k=2)

In [132]:
# Hotel ratio test
goodHl = []
for m in matchesHl:
    if m[0].distance < 0.5*m[1].distance:
        goodHl.append(m)
matchesHl = np.asarray(goodHl)
# Shanghai ratio test
goodSh = []
for m in matchesSh:
    if m[0].distance < 0.5*m[1].distance:
        goodSh.append(m)
matchesSh = np.asarray(goodSh)

In [133]:
#Hotel homography matrix
if len(matches[:,0]) >= 4:
    srcHl = np.float32([ kpHl1[m.queryIdx].pt for m in matchesHl[:,0] ]).reshape(-1,1,2)
    finalHl = np.float32([ kpHl2[m.trainIdx].pt for m in matchesHl[:,0] ]).reshape(-1,1,2)
    HHl, masked = cv2.findHomography(srcHl, finalHl, cv2.RANSAC, 5.0)
    #print H
else:
    raise AssertionError("Can’t find enough keypoints hl.")
#Shanghai homography matrix
if len(matches[:,0]) >= 4:
    srcSh = np.float32([ kpSh1[m.queryIdx].pt for m in matchesSh[:,0] ]).reshape(-1,1,2)
    finalSh = np.float32([ kpSh2[m.trainIdx].pt for m in matchesSh[:,0] ]).reshape(-1,1,2)
    HSh, masked = cv2.findHomography(srcSh, finalSh, cv2.RANSAC, 5.0)
else:
    raise AssertionError("Can’t find enough keypoints Sh.")

In [135]:
finalHl = cv2.warpPerspective(imgHlR,HHl,(imgHlL.shape[1] + imgHlR.shape[1], imgHlL.shape[0]))
finalHl[0:imgHlL.shape[0], 0:imgHlL.shape[1]] = imgHlL
draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = None,
                   flags = 2)
matchesHl = cv2.drawMatchesKnn(imgHlR,kpHl1,imgHlL,kpHl2,goodHl,None,**draw_params)

finalSh = cv2.warpPerspective(imgShR,HSh,(imgShL.shape[1] + imgShR.shape[1], imgShL.shape[0]))
finalSh[0:imgShL.shape[0], 0:imgShL.shape[1]] = imgShL
matchesSh = cv2.drawMatchesKnn(imgShR,kpSh1,imgShL,kpSh2,goodSh,None,**draw_params)

cv2.imwrite("matchShangHai/SIFTmatches.png",matchesSh)
cv2.imwrite("panShangHai/SIFTpan.png",finalSh)
cv2.imwrite("matchesHotel/SIFTmatches.png",matchesHl)
cv2.imwrite("panHotel/SIFTpan.png",finalHl)

True